In [1]:
import numpy as np
import nibabel as nib
import os
import csv
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, Subtract
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input
#from patchify import patchify, unpatchify
from sklearn.model_selection import train_test_split
from numpy.random import rand
from random import seed
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [2]:
def getModel(sigLength):
    input_layer = Input(shape=(sigLength, 1))

    # Layer 1 - Convolutional Layer + ReLU activation
    x = Conv1D(filters=16, kernel_size=16, kernel_initializer='Orthogonal',
               padding='same', name=f'Conv1')(input_layer)
    x = Activation('relu', name=f'ReLU1')(x)
    max_pool_1d = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding='valid')
    pool1 = max_pool_1d(x)
    x = Conv1D(filters=32, kernel_size=8, padding='same', name=f'Conv2')(pool1)
    x = Activation('relu', name=f'ReLU2')(x)
    pool2 = max_pool_1d(x)
    pool2_flat = tf.keras.layers.Flatten()(pool2)
    logits = tf.keras.layers.Dense(units=sigLength, activation='relu')(pool2_flat)
    return Model(inputs=input_layer, outputs=logits)


In [3]:
sigLength = 184     # length of DWI signal
tmp = nib.load('nodif_brain_mask.nii.gz')
msk_img = tmp.get_fdata()
sz = msk_img.shape
numVol = np.int(sum(msk_img.ravel()))
msk = np.reshape(msk_img, sz[0]*sz[1]*sz[2])
# the normal noised data
tmp = nib.load('dwi_gt184.nii.gz')
normal_img = tmp.get_fdata()
sz = normal_img.shape
imgRaw = np.reshape(normal_img, (sz[0]*sz[1]*sz[2], sz[3]))
sigRaw = np.delete(imgRaw, np.where(msk != 1), axis=0)
learning_rate = 0.001
num_epochs = 2000
batch_size = 6000
# High-noise data
tmp = nib.load('dwi_gt184snr30.nii.gz')
imgNoised = np.reshape(tmp.get_fdata(), (sz[0]*sz[1]*sz[2], sz[3]))
sigNoised = np.delete(imgNoised, np.where(msk != 1), axis=0)
print(sigNoised.shape)

(76766, 184)


In [4]:
model = getModel(sigLength)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 184, 1)]     0                                            
__________________________________________________________________________________________________
Conv1 (Conv1D)                  (None, 184, 16)      272         input_1[0][0]                    
__________________________________________________________________________________________________
ReLU1 (Activation)              (None, 184, 16)      0           Conv1[0][0]                      
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    multiple             0           ReLU1[0][0]                      
                                                                 ReLU2[0][0]                  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sigNoised, sigRaw)
print(X_train.shape)
model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_data=(X_test, y_test))
model.save('1D_CNN_denoise_model')

(57574, 184)
Epoch 1/2000


In [ ]:
from keras.models import load_model
my_model = load_model('1D_CNN_denoise_model', compile=False)
predicted_img = my_model.predict(imgNoised)
print(predicted_img.shape)
x_img = np.float32( np.reshape(predicted_img, (sz[0],sz[1],sz[2],sz[3])) )
img_denoised = x_img
print(img_denoised.shape)
for ii in range(0,sigLength):
    img_denoised[:,:,:,ii] = img_denoised[:,:,:,ii] * msk_img
# save the denoised result to a nifti
x1 = nib.Nifti1Image(img_denoised, tmp.affine, tmp.header)
nib.save(x1,'Denoised.nii.gz')